# 状態空間モデルとカルマンフィルタ

- モデル: 線形ガウス状態空間モデル
- 状態の推定方法: カルマンフィルタor散漫カルマンフィルタ、平滑化
- パラメタ推定の方法: 最尤法
- ソフト:　R
- 使用するパッケージ: dlm, KFAS(推奨)

**線形かつ正規分布(ガウス分布)** という制約のついた状態空間モデルについて考えてみる。

## 線形ガウス状態空間モデルのデータ表現

$
\begin{align}
    x_t &= T_t x_{t-1} + R_t \xi_t, \quad &\xi_t \sim N(0, Q_t) \\
    y_t &= Z_t x_t + \varepsilon_t, \quad &\varepsilon_t \sim N(0, H_t)
\end{align}
$

$
\begin{align}
    状態 &= 前時点の状態を用いた予測値 + 過程誤差　\\
    観測地 &= 状態 + 観測誤差
\end{align}
$

- $x_t$は$t$時点の状態で$k$次元のベクトルをとる
- $y_t$は$t$時点の観測値
- $T_t$, $R_t$, $Z_t$はモデルの表現形式を決める行列

<img src="pics\KalmanFilter_ja.png">

## 補正の考え方

観測データから状態を補正したい場合、**過程誤差** と **観測誤差** の分散の大きさを気にして補正する必要がある。  
過程誤差と観測誤差の大きさの比率を勘案しつつ、**フィルタリング(状態の補正)** を行う。  
過程誤差の大きさと観測誤差の大きさというパラメタを、最尤法と呼ばれる方法を用いて推定する。  

- 過程誤差の分散が大きい場合
    - 未来の状態(予測)が当たり難い
        - 「観測値を用いた状態の補正」をすべき


- 観測誤差の分散が大きい場合
    - 観測値があまり信用できない
        - 「観測値を用いた状態の補正」はあまり行わない方が良い

## 線形ガウス状態空間モデルを推定する流れ

1. 状態方程式・観測方程式を用いてモデルの構造を表現する
2. 「とりあえず」パラメタを使ってフィルタリングする
3. カルマンフィルタの結果を援用し、最尤法を用いてパラメタを推定する
4. 推定されたパラメタを用いて、再度フィルタリングを行う
5. (必要ならば)推定されたパラメタを用いて、平滑化を行う

- 誤差が正規分布に従っていることを前提としたモデルである

## 表現: 状態方程式・観測方程式による表現技法

データを人間が理解できる表現形式に落とし込みたい

## ローカルレベルモデル

過程誤差と観測誤差をともに認めたモデル。

$
\begin{align}
    \mu_t &= \mu_{t-1} + w_t, \quad &w_t \sim N(0, \sigma_w^2) \\
    y_t &= \mu_t + \upsilon_t, \quad &\upsilon_t \sim N(0, \sigma_\upsilon^2)
\end{align}
$

状態の繊維はホワイトノイズの累積和なのでランダムウォーク系列になっている。  
これに観測誤差が加わるのでランダムウォーク・プラス・ノイズモデルとも呼ばれる。

$\sigma_w^2$と$\sigma_\upsilon^2$を未知パラメタとして推定する必要がある。

### ローカルレベルモデルによる予測
t-1時点の状態$\mu_{t-1}$が定まった条件におけるt時点の予測値(条件付き期待値)は以下のようになる。  

$
    E(\mu_t|\mu_{t-1}) = \mu_{t-1}
$

- 過程誤差の期待値は0なので消えてなくなる。
    - 予測された観測地 $y_t$ も $\mu_{t-1}$ となる。
        - ローカルレベルモデルの予測は観測値と同じになるなるので将来予測ではあまり役にたたない
        - ローカルレベルモデルは状態空間モデルの出発点だとみなす

## ローカル線形トレンドモデル

ローカルレベルモデルにトレンド成分を導入する。  
ローカル線形トレンドモデルの状態・観測方程式は以下のようになる。  

$
\begin{align}
    \delta_t &= \delta_{t-1} + \zeta_t, \quad &\zeta_t &\sim N(0, \sigma_\zeta^2) \\
    \mu_t &= \mu_{t-1} + \delta_{t-1} + w_t, \quad &w_t &\sim N(0, \sigma_w^2) \\
    y_t &= \mu_t + \upsilon_t, \quad &\upsilon_t &\sim N(0, \sigma_\upsilon^2)
\end{align}
$

- トレンド成分が時間によって変化することを想定した時系列モデル
- トレンド成分はホワイトノイズの累積和、ランダムウォーク系列で近似できると想定している
- ランダムウォーク系列の予測値(期待値)は1時点前の値そのもの。

- カルマンフィルタにより、観測地が得られるたびにトレンドの値も補正が入る。
- データが得られるたびにトレンド成分を更新していき、「修正済みの最新トレンド成分」に基づいて将来の予測を行う。

## 周期的変動のモデル化
$
\begin{align}
    \gamma_{1,t} &= - \gamma_{1,t-1}-\gamma_{2,t-1} - \gamma_{3,t-1} + \eta_t, \quad &\eta_t &\sim N(0, 
\sigma_\eta^2) \\
    \gamma_{2,t} &= \gamma_{1,t-1} \\
    \gamma_{3,t} &= \gamma_{2,t-1} \\
    y_t &= \gamma_{1,t} + \upsilon_t, \quad &\upsilon_t &\sim N(0,\sigma_\upsilon^2) \\
\end{align}
$

## カルマンフィルタ

- カルマンフィルタは「状態の1時先予測」と「観測値を用いた状態の補正」を交互に繰り返していくもの。  
- ロカールレベルモデルにおける「状態の1時先予測」は「1時点前の状態と同じ」でした。  
- 「観測値を用いた状態の補正」について考える

**補正後の状態(フィルタ化推定量)**  
**= 補正前の状態 + カルマンゲイン(1以下) × (実際の観測値 - 予測された観測値)**  

外れた分よりもすくなめに補正していることがわかる。  

## カルマンゲインの求め方

- 状態の予測誤差が大きいとき、補正する量も大きくするべき
- 観測誤差が大きいとき、観測地を用いた補正量は小さくするべき

$
    カルマンゲイン = \frac{状態の予測誤差の分散}{状態の予測誤差の分散 + 観測誤差の分散}
$